In [140]:
# Import subjects

import pickle

para = 'Rest'

# Load meta dict

with open('../../AllSubjectsMeta.bin', 'rb') as f:
    meta = pickle.load(f)
    
# Load rest subject ids and splits

with open('../../../BrainAgePredictWrat/' + para + 'Only10FoldSplit.bin', 'rb') as f:
    splits = pickle.load(f)
    subids = splits['cand' + para + 'YesWrat']
    groups = splits['groups']
    
print(len(subids))

804


In [141]:
import numpy as np

# Get feature vectors (including true ages)

# Create feature vectors (right now just ages, maleness, and femaless)

X_all = []
for subid in subids:
    subj = meta[subid]
    maleness = 1 if subj['meta']['Gender'] == 'M' else 0
    femaleness = 1 if maleness == 0 else 0
    feat = np.array([subj['meta']['AgeInMonths'], maleness, femaleness])
    X_all.append(feat)
X_all = np.vstack(X_all)

print(X_all[10:20])
print('Complete')

[[223   1   0]
 [190   0   1]
 [197   0   1]
 [145   1   0]
 [148   0   1]
 [142   0   1]
 [123   1   0]
 [176   1   0]
 [129   0   1]
 [173   1   0]]
Complete


In [142]:
# Load WRAT for subjects

import numpy as np

idWratDict = {}

with open('../../wrat.csv') as f:
    for line in f.readlines()[1:]:
        parts = line.split(',')
        try:
            idWratDict[int(parts[0])] = int(parts[3])
        except Exception as e:
            print(parts[0])
            print(e)
            
wrat = np.array([idWratDict[subid] for subid in subids])

print(wrat.shape)
print('Complete')

603056110671
invalid literal for int() with base 10: 'NaN\n'
603317055381
invalid literal for int() with base 10: 'NaN\n'
604847970437
invalid literal for int() with base 10: 'NaN\n'
(804,)
Complete


In [147]:
# Baseline

stat = wrat - np.mean(wrat)
# stat = agesTrue - np.mean(agesTrue)
print(stat.shape)

# stat = (np.sum(stat**2)/len(stat))**0.5
stat = np.sum(np.abs(stat))/len(stat)
print(stat)

stats = []
for trainIdcs, testIdcs in groups:
    stat = wrat[testIdcs] - np.mean(wrat[testIdcs])
#     stat = agesTrue[testIdcs] - np.mean(agesTrue[testIdcs])
#     stat = (np.sum(stat**2)/len(testIdcs))**0.5
    stat = np.sum(np.abs(stat))/len(testIdcs)
    stats.append(stat)
    
print(np.std(stats))

(804,)
12.405902700428207
1.5889835088613964


In [143]:
# Get all ages, true and predicted

para = 'Rest'

agesTrue = X_all[:,0]

with open('../../../BrainAgePredictWrat/Wrat/LeastSquares' + para + 'WratPredictions.bin', 'rb') as f:
    agesPred = pickle.load(f)
    
with open('../../../BrainAgePredictWrat/Wrat/LeastSquares5' + para + 'WratPredictions.bin', 'rb') as f:
    agesPred5 = pickle.load(f)
    
with open('../../../BrainAgePredictWrat/Wrat/CNNWrat' + para + 'Predictions.bin', 'rb') as f:
    agesPredCNN = pickle.load(f)
    
with open('../../../BrainAgePredictWrat/Wrat/MLPWrat' + para + 'Predictions.bin', 'rb') as f:
    agesPredMLP = pickle.load(f)
    
with open('../../../BrainAgePredictWrat/Wrat/TransformerWrat' + para + 'Predictions.bin', 'rb') as f:
    agesPredTrans = pickle.load(f)
    
print('Complete')

Complete


In [139]:
stats = []
for trainIdcs, testIdcs in groups:
#     stat = np.corrcoef(agesPredTrans[testIdcs], wrat[testIdcs])[0,1]
#     stat = np.corrcoef(agesPredTrans[testIdcs], agesTrue[testIdcs])[0,1]
    stat = agesPred[testIdcs]-wrat[testIdcs]
#     stat = (np.sum(stat**2)/len(testIdcs))**0.5
    stat = np.sum(np.abs(stat))/len(testIdcs)
    stats.append(stat)
    
print(stats)
print(np.std(stats))

[11.919642120738223, 12.065902998521663, 11.917443779241312, 14.020656212257167, 11.815115138119257, 11.215549597750814, 11.736728419558052, 10.140400560612736, 12.346745722089967, 11.206884890044625]
0.9362744575813814
